In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import asc, col, isnan, when, count, median, udf, concat, month, year, substring, lit
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier, OneVsRest
from pyspark.ml import Pipeline
import os
import pyarrow

In [2]:
#Create a spark session and read the data
conf = SparkConf() \
    .setAppName("data_cleaning") \
    .set("spark.driver.memory", "15g")\
    .set("spark.executor.cores","8") \
    .set("spark.sql.execution.arrow.pyspark.enabled","true")

spark = SparkSession.builder.config(conf=conf).getOrCreate()

csv_file_path ="/workspace/data.csv"
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
df.orderBy(asc("fecha_dato"))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/23 14:46:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


DataFrame[fecha_dato: date, ncodpers: double, ind_empleado: string, pais_residencia: string, sexo: string, age: string, fecha_alta: date, ind_nuevo: string, antiguedad: string, indrel: string, ult_fec_cli_1t: date, indrel_1mes: string, tiprel_1mes: string, indresi: string, indext: string, conyuemp: string, canal_entrada: string, indfall: string, tipodom: string, cod_prov: string, nomprov: string, ind_actividad_cliente: string, renta: double, segmento: string, ind_ahor_fin_ult1: int, ind_aval_fin_ult1: int, ind_cco_fin_ult1: int, ind_cder_fin_ult1: int, ind_cno_fin_ult1: int, ind_ctju_fin_ult1: int, ind_ctma_fin_ult1: int, ind_ctop_fin_ult1: int, ind_ctpp_fin_ult1: int, ind_deco_fin_ult1: int, ind_deme_fin_ult1: int, ind_dela_fin_ult1: int, ind_ecue_fin_ult1: int, ind_fond_fin_ult1: int, ind_hip_fin_ult1: int, ind_plan_fin_ult1: int, ind_pres_fin_ult1: int, ind_reca_fin_ult1: int, ind_tjcr_fin_ult1: int, ind_valo_fin_ult1: int, ind_viv_fin_ult1: int, ind_nomina_ult1: string, ind_nom_pen

# Xử lý cột "age"

Đoạn mã này làm sạch và gán giá trị cho cột "age" trong DataFrame PySpark, bao gồm:

1.  **Tính tuổi trung bình:**
    *   `mean_age_18_to_30`: Trung bình tuổi từ 18 đến 30 (bao gồm cả hai đầu).
    *   `mean_age_30_to_100`: Trung bình tuổi từ 30 đến 100 (bao gồm cả hai đầu).
    *   `overall_mean_age`: Trung bình tuổi của toàn bộ DataFrame.

    Sử dụng `df.filter()`, `F.col()`, `F.mean()`, và `.collect()[0][0]` để lấy giá trị.  Lưu ý: `.collect()` đưa dữ liệu về driver, cẩn thận với DataFrame lớn.

2.  **Xử lý ngoại lai:**
    *   Thay thế tuổi `< 18` bằng `mean_age_18_to_30`.
    *   Thay thế tuổi `> 100` bằng `mean_age_30_to_100`.
    *   Giữ nguyên tuổi từ 18-100.

    Dùng `df.withColumn()`, `F.when()`, và `.otherwise()`.

3.  **Điền giá trị thiếu (NA/null):**
    *   Điền giá trị thiếu trong cột "age" bằng `overall_mean_age`.
    *   Sử dụng `df.fillna()`.

4.  **Chuyển kiểu dữ liệu:**
    *   Chuyển cột "age" sang kiểu số nguyên (int).
    *    Dùng `F.col("age").cast("int")`

In [3]:
#Fill NA with overall mean
mean_age_18_to_30 = df.filter((F.col("age") >= 18) & (F.col("age") <= 30)).select(F.mean("age")).collect()[0][0]
mean_age_30_to_100 = df.filter((F.col("age") >= 30) & (F.col("age") <= 100)).select(F.mean("age")).collect()[0][0]
overall_mean_age = df.select(F.mean("age")).collect()[0][0]

df = df.withColumn(
    "age",
    F.when(F.col("age") < 18, mean_age_18_to_30)
     .when(F.col("age") > 100, mean_age_30_to_100)
     .otherwise(F.col("age"))
)

df = df.fillna({"age": overall_mean_age})
df = df.withColumn("age", F.col("age").cast("int"))

# df = df.withColumn(
#     "age",
#     F.when(F.col("age").isNull(), overall_mean_age).otherwise(F.col("age"))
# )

# Điền giá trị thiếu (Missing Value Imputation)

Đoạn mã này thực hiện điền các giá trị thiếu (NA/null) trong các cột khác nhau của một DataFrame PySpark (`df`). Mỗi dòng sử dụng `df.fillna()` để điền giá trị thiếu trong một cột cụ thể bằng một giá trị cố định(trong trường hợp này là giá trị xuất hiện nhiều nhất).

In [ ]:
# #Filling missing falue with the most common status
df = df.fillna({"ind_nuevo": 1})
df = df.fillna({"indrel": 1})
df = df.fillna({"indfall": "N"})
df = df.fillna({"tiprel_1mes": "A"})
#Fill ult_fec_cli_1t with a value in the future to indicate that they are still a primary customer
df = df.fillna({"ult_fec_cli_1t" : '2020-01-01'})
df = df.fillna({"conyuemp" : "-1"})

# Chuyển đổi kiểu dữ liệu sang số nguyên (IntegerType) trong PySpark

Đoạn mã PySpark này thực hiện chuyển đổi kiểu dữ liệu của một danh sách các cột trong DataFrame (`df`) sang kiểu số nguyên (`IntegerType`).  Điều này thường cần thiết sau khi thực hiện các thao tác như điền giá trị thiếu (imputation), vì các giá trị được điền có thể có kiểu dữ liệu khác (ví dụ: float sau khi điền bằng giá trị trung bình).


In [6]:
#Cast non-integer type to integer type

# List of columns to cast
columns_to_cast = [
    "ind_nuevo",
    "indrel",
    "ind_ahor_fin_ult1",
    "ind_aval_fin_ult1",
    "ind_cco_fin_ult1",
    "ind_cder_fin_ult1",
    "ind_cno_fin_ult1",
    "ind_ctju_fin_ult1",
    "ind_ctma_fin_ult1",
    "ind_ctop_fin_ult1",
    "ind_ctpp_fin_ult1",
    "ind_deco_fin_ult1",
    "ind_deme_fin_ult1",
    "ind_dela_fin_ult1",
    "ind_ecue_fin_ult1",
    "ind_fond_fin_ult1",
    "ind_hip_fin_ult1",
    "ind_plan_fin_ult1",
    "ind_pres_fin_ult1",
    "ind_reca_fin_ult1",
    "ind_tjcr_fin_ult1",
    "ind_valo_fin_ult1",
    "ind_viv_fin_ult1",
    "ind_nomina_ult1",
    "ind_nom_pens_ult1",
    "ind_recibo_ult1",
    "renta"
]

# Cast each column to IntegerType
for column in columns_to_cast:
    df = df.withColumn(column, df[column].cast(IntegerType()))

# Điền giá trị thiếu bằng giá trị trung vị (Median)

Đoạn mã này thực hiện điền các giá trị thiếu (NA/null) của DataFrame PySpark (`df`) bằng giá trị *trung vị* của cột đó.

In [ ]:
#Fill with median value
window_spec = Window.orderBy(F.col("fecha_alta"))
dates = df.select(
    "fecha_alta",
    F.row_number().over(window_spec).alias("index")
)
total_rows = dates.count()
median_index = (total_rows // 2) + 1 
median_value = dates.filter(F.col("index") == median_index).select("fecha_alta").collect()[0][0]
df = df.withColumn(
    "fecha_alta",
    F.when(F.col("fecha_alta").isNull(), median_value).otherwise(F.col("fecha_alta"))

median_value = df.select(F.median("ind_actividad_cliente")).collect()[0][0]

df = df.withColumn(
    "ind_actividad_cliente",
    F.when(F.col("ind_actividad_cliente").isNull(), median_value).otherwise(F.col("ind_actividad_cliente"))
)
)

#Fill with median of province
grouped = df.groupBy("nomprov").agg(median("renta").alias("renta_median"))

df = df.join(grouped, "nomprov", "left")

df = df.withColumn(
    "renta",
    F.when(F.col("renta").isNull(), col("renta_median")).otherwise(F.col("renta"))
)

25/02/23 14:52:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/23 14:52:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/23 14:52:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/23 14:52:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/23 14:52:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


# Loại bỏ các hàng chứa giá trị NULL trong PySpark

Đoạn mã PySpark này thực hiện loại bỏ các hàng chứa giá trị NULL (hay NA - Not Available) trong các cột cụ thể của DataFrame (`df`). Việc loại bỏ hàng có giá trị NULL là một cách xử lý dữ liệu thiếu, thường được thực hiện khi số lượng giá trị thiếu không quá lớn và việc loại bỏ chúng không ảnh hưởng đáng kể đến kết quả phân tích.

In [ ]:
#Drop rows that have NULL value
# df = df.na.drop(subset=["nomprov"]) 
df = df.na.drop(subset=["ind_nom_pens_ult1"]) 
df = df.na.drop(subset=["ind_nomina_ult1"]) 

# Ánh xạ giá trị chuỗi sang số nguyên (String to Integer Mapping) trong PySpark

Đoạn mã PySpark này thực hiện chuyển đổi các giá trị chuỗi trong một số cột của DataFrame (`df`) thành các giá trị số nguyên tương ứng.  Việc này thường được thực hiện khi dữ liệu gốc chứa các giá trị dạng text (ví dụ: mã quốc gia, loại khách hàng) nhưng mô hình machine learning lại yêu cầu đầu vào là số.  Quá trình này được gọi là *label encoding*.

In [12]:
#Map a string value to a integer value
canal_dict = {'KAI': 35,'KBG': 17,'KGU': 149,'KDE': 47,'KAJ': 41,'KCG': 59,
 'KHM': 12,'KAL': 74,'KFH': 140,'KCT': 112,'KBJ': 133,'KBL': 88,'KHQ': 157,'KFB': 146,'KFV': 48,'KFC': 4,
 'KCK': 52,'KAN': 110,'KES': 68,'KCB': 78,'KBS': 118,'KDP': 103,'KDD': 113,'KBX': 116,'KCM': 82,
 'KAE': 30,'KAB': 28,'KFG': 27,'KDA': 63,'KBV': 100,'KBD': 109,'KBW': 114,'KGN': 11,
 'KCP': 129,'KAK': 51,'KAR': 32,'KHK': 10,'KDS': 124,'KEY': 93,'KFU': 36,'KBY': 111,
 'KEK': 145,'KCX': 120,'KDQ': 80,'K00': 50,'KCC': 29,'KCN': 81,'KDZ': 99,'KDR': 56,
 'KBE': 119,'KFN': 42,'KEC': 66,'KDM': 130,'KBP': 121,'KAU': 142,'KDU': 79,
 'KCH': 84,'KHF': 19,'KCR': 153,'KBH': 90,'KEA': 89,'KEM': 155,'KGY': 44,'KBM': 135,
 'KEW': 98,'KDB': 117,'KHD': 2,'RED': 8,'KBN': 122,'KDY': 61,'KDI': 150,'KEU': 72,
 'KCA': 73,'KAH': 31,'KAO': 94,'KAZ': 7,'004': 83,'KEJ': 95,'KBQ': 62,'KEZ': 108,
 'KCI': 65,'KGW': 147,'KFJ': 33,'KCF': 105,'KFT': 92,'KED': 143,'KAT': 5,'KDL': 158,
 'KFA': 3,'KCO': 104,'KEO': 96,'KBZ': 67,'KHA': 22,'KDX': 69,'KDO': 60,'KAF': 23,'KAW': 76,
 'KAG': 26,'KAM': 107,'KEL': 125,'KEH': 15,'KAQ': 37,'KFD': 25,'KEQ': 138,'KEN': 137,
 'KFS': 38,'KBB': 131,'KCE': 86,'KAP': 46,'KAC': 57,'KBO': 64,'KHR': 161,'KFF': 45,
 'KEE': 152,'KHL': 0,'007': 71,'KDG': 126,'025': 159,'KGX': 24,'KEI': 97,'KBF': 102,
 'KEG': 136,'KFP': 40,'KDF': 127,'KCJ': 156,'KFR': 144,'KDW': 132,-1: 6,'KAD': 16,
 'KBU': 55,'KCU': 115,'KAA': 39,'KEF': 128,'KAY': 54,'KGC': 18,'KAV': 139,'KDN': 151,
 'KCV': 106,'KCL': 53,'013': 49,'KDV': 91,'KFE': 148,'KCQ': 154,'KDH': 14,'KHN': 21,
 'KDT': 58,'KBR': 101,'KEB': 123,'KAS': 70,'KCD': 85,'KFL': 34,'KCS': 77,'KHO': 13,
 'KEV': 87,'KHE': 1,'KHC': 9,'KFK': 20,'KDC': 75,'KFM': 141,'KHP': 160,'KHS': 162,
 'KFI': 134,'KGV': 43}


pais_dict = {'LV': 102,'CA': 2,'GB': 9,'EC': 19,'BY': 64,'ML': 104,'MT': 118,
 'LU': 59,'GR': 39,'NI': 33,'BZ': 113,'QA': 58,'DE': 10,'AU': 63,'IN': 31,
 'GN': 98,'KE': 65,'HN': 22,'JM': 116,'SV': 53,'TH': 79,'IE': 5,'TN': 85,
 'PH': 91,'ET': 54,'AR': 13,'KR': 87,'GA': 45,'FR': 8,'SG': 66,'LB': 81,
 'MA': 38,'NZ': 93,'SK': 69,'CN': 28,'GI': 96,'PY': 51,'SA': 56,'PL': 30,
 'PE': 20,'GE': 78,'HR': 67,'CD': 112,'MM': 94,'MR': 48,'NG': 83,'HU': 106,
 'AO': 71,'NL': 7,'GM': 110,'DJ': 115,'ZA': 75,'OM': 100,'LT': 103,'MZ': 27,
 'VE': 14,'EE': 52,'CF': 109,'CL': 4,'SL': 97,'DO': 11,'PT': 26,'ES': 0,
 'CZ': 36,'AD': 35,'RO': 41,'TW': 29,'BA': 61,'IS': 107,'AT': 6,'ZW': 114,
 'TR': 70,'CO': 21,'PK': 84,'SE': 24,'AL': 25,'CU': 72,'UY': 77,'EG': 74,'CR': 32,
 'GQ': 73,'MK': 105,'KW': 92,'GT': 44,'CM': 55,'SN': 47,'KZ': 111,'DK': 76,
 'LY': 108,'AE': 37,'PA': 60,'UA': 49,'GW': 99,'TG': 86,'MX': 16,'KH': 95,
 'FI': 23,'NO': 46,'IT': 18,'GH': 88, 'JP': 82,'RU': 43,'PR': 40,'RS': 89,
 'DZ': 80,'MD': 68,-1: 1,'BG': 50,'CI': 57,'IL': 42,'VN': 90,'CH': 3,'US': 15,'HK': 34,
 'CG': 101,'BO': 62,'BR': 17,'BE': 12,'BM': 117}

emp_dict = {'N':0,-1:-1,'A':1,'B':2,'F':3,'S':4}
indfall_dict = {'N':0,-1:-1,'S':1}
sexo_dict = {'V':0,'H':1,-1:-1}
tiprel_dict = {'A':0,-1:-1,'I':1,'P':2,'N':3,'R':4}
indresi_dict = {'N':0,-1:-1,'S':1}
indext_dict = {'N':0,-1:-1,'S':1}
conyuemp_dict = {'N':0,-1:-1,'S':1}
segmento_dict = {-1:-1,'01 - TOP':1,'02 - PARTICULARES':2,'03 - UNIVERSITARIO':3}

spark_df = df
def create_mapping_udf(mapping_dict):
    def map_value(value):
        return mapping_dict.get(value, -1)  # Default to -1 if not found
    return udf(map_value, IntegerType())

canal_udf = create_mapping_udf(canal_dict)
pais_udf = create_mapping_udf(pais_dict)
indfall_udf = create_mapping_udf(indfall_dict)
sexo_udf = create_mapping_udf(sexo_dict)
tiprel_udf = create_mapping_udf(tiprel_dict)
indresi_udf = create_mapping_udf(indresi_dict)
indext_udf = create_mapping_udf(indext_dict)
conyuemp_udf = create_mapping_udf(conyuemp_dict)
segmento_udf = create_mapping_udf(segmento_dict)
emp_udf = create_mapping_udf(emp_dict)

def apply_udfs(spark_df):
    spark_df = spark_df.withColumn("canal_entrada", canal_udf(col("canal_entrada")).cast(IntegerType()))
    spark_df = spark_df.withColumn("pais_residencia", pais_udf(col("pais_residencia")).cast(IntegerType()))
    spark_df = spark_df.withColumn("indfall", indfall_udf(col("indfall")).cast(IntegerType()))
    spark_df = spark_df.withColumn("sexo", sexo_udf(col("sexo")).cast(IntegerType()))
    spark_df = spark_df.withColumn("tiprel_1mes", tiprel_udf(col("tiprel_1mes")).cast(IntegerType()))
    spark_df = spark_df.withColumn("indresi", indresi_udf(col("indresi")).cast(IntegerType()))
    spark_df = spark_df.withColumn("indext", indext_udf(col("indext")).cast(IntegerType()))
    spark_df = spark_df.withColumn("conyuemp", conyuemp_udf(col("conyuemp")).cast(IntegerType()))
    spark_df = spark_df.withColumn("segmento", segmento_udf(col("segmento")).cast(IntegerType()))
    spark_df = spark_df.withColumn("ind_empleado", segmento_udf(col("ind_empleado")).cast(IntegerType()))
    
    return spark_df

spark_df = apply_udfs(spark_df)

# Xử lý và tách cột ngày tháng (Date) trong PySpark

Đoạn mã PySpark này thực hiện các thao tác sau trên các cột kiểu ngày tháng (Date) trong DataFrame (`spark_df`):

1.  **Trích xuất thành phần ngày tháng:** Tách các thành phần năm, tháng, và ngày từ các cột ngày tháng gốc (`fecha_dato`, `ult_fec_cli_1t`, `fecha_alta`) và tạo các cột mới tương ứng.
2.  **Xử lý giá trị NULL:** Điền giá trị `-1` vào bất kỳ giá trị NULL nào xuất hiện trong các cột mới tạo.
3.  **Xóa cột gốc:** Loại bỏ các cột ngày tháng gốc sau khi đã trích xuất thông tin.
4. **Tính tháng dạng số nguyên**: Tạo ra một cột tháng dạng số nguyên bằng cách lấy tháng cộng với 12 lần năm.

In [ ]:
#Three cells bellow format DATE type into month,day,year,*int and drop the DATE column
spark_df = spark_df.withColumn("fecha_dato_month", substring("fecha_dato", 6, 2).cast(IntegerType()))
spark_df = spark_df.withColumn("fecha_dato_year", (substring("fecha_dato", 1, 4).cast(IntegerType()) - 2015))
spark_df = spark_df.withColumn("month_int", (col("fecha_dato_month") + 12 * col("fecha_dato_year")).cast(IntegerType()))
spark_df = spark_df.withColumn("fecha_dato_day", substring("fecha_dato", 9, 2).cast(IntegerType()))
for col_name in ["fecha_dato_month", "fecha_dato_year", "month_int", "fecha_dato_day"]:
    spark_df = spark_df.withColumn(col_name, \
                        when(col(col_name).isNull(), lit(-1)) \
                        .otherwise(col(col_name)))

# Drop the original column
spark_df = spark_df.drop("fecha_dato")
spark_df = spark_df.withColumn("ult_fec_cli_1t_month", substring("ult_fec_cli_1t", 6, 2).cast(IntegerType()))
spark_df = spark_df.withColumn("ult_fec_cli_1t_year", (substring("ult_fec_cli_1t", 1, 4).cast(IntegerType()) - 2015))
spark_df = spark_df.withColumn("ult_fec_cli_1t_day", substring("ult_fec_cli_1t", 9, 2).cast(IntegerType()))
spark_df = spark_df.withColumn("ult_fec_cli_1t_month_int", (col("ult_fec_cli_1t_month") + 12 * col("ult_fec_cli_1t_year")))
    #Check if any value is null, then fill with -1
for col_name in ["ult_fec_cli_1t_month", "ult_fec_cli_1t_year", "ult_fec_cli_1t_day", "ult_fec_cli_1t_month_int"]:
    spark_df = spark_df.withColumn(col_name, \
                       when(col(col_name).isNull(), -1) \
                       .otherwise(col(col_name)))
spark_df = spark_df.drop("ult_fec_cli_1t")
spark_df = spark_df.withColumn("fecha_alta_month", substring("fecha_alta", 6, 2).cast(IntegerType()))
spark_df = spark_df.withColumn("fecha_alta_year", (substring("fecha_alta", 1, 4).cast(IntegerType()) - 2015))
spark_df = spark_df.withColumn("fecha_alta_day", substring("fecha_alta", 9, 2).cast(IntegerType()))
spark_df = spark_df.withColumn("fecha_alta_month_int", (col("fecha_alta_month") + 12 * col("fecha_alta_year")).cast(IntegerType()))

# Drop the original column
spark_df = spark_df.drop("fecha_alta")

# Xử lý giá trị thiếu và chuyển đổi kiểu dữ liệu trong PySpark

Đoạn mã PySpark này thực hiện một loạt các thao tác trên DataFrame (`spark_df`) để:

1.  **Điền giá trị thiếu (NULL):** Thay thế các giá trị NULL bằng `-1` trong một số cột.
2.  **Chuyển đổi kiểu dữ liệu:** Chuyển đổi kiểu dữ liệu của một số cột sang số nguyên (`IntegerType`).
3.  **Thay thế giá trị:** Thay giá trị "P" bằng -2 trong cột `indrel_1mes`
4.  **Xóa cột:** Loại bỏ cột `nomprov`.


In [ ]:
#Fill NULL value with -1 and cast to integer type
spark_df = spark_df.withColumn("indrel_1mes",when(col("indrel_1mes") == "P", -2).otherwise(col("indrel_1mes")))
spark_df = spark_df.fillna({"indrel_1mes": -1})
spark_df = spark_df.withColumn("ind_actividad_cliente", when(col("ind_actividad_cliente").isNull(), lit(-1)).otherwise(col("ind_actividad_cliente").cast(IntegerType())))
spark_df = spark_df.withColumn("indrel_1mes", col("indrel_1mes").cast(IntegerType()))
spark_df = spark_df.withColumn("tipodom", when(col("tipodom").isNull(), lit(-1)).otherwise(col("tipodom").cast(IntegerType())))
spark_df = spark_df.withColumn("cod_prov", when(col("cod_prov").isNull(), lit(-1)).otherwise(col("cod_prov").cast(IntegerType())))
spark_df = spark_df.withColumn("antiguedad",when(col("antiguedad").isNull(), lit(-1)).otherwise(col("antiguedad")))
spark_df = spark_df.withColumn("antiguedad", col("antiguedad").cast(IntegerType()))
spark_df = spark_df.drop("nomprov")

In [ ]:
#Count the number of NULLs value in dataframe
# Dict_Null = {col:spark_df.filter(spark_df[col].isNull()).count() for col in spark_df.columns}
# Dict_Null

# Tạo độ trễ (Lag Features) trong PySpark

Đoạn mã PySpark này tạo các *đặc trưng trễ* (lag features) cho một tập hợp các cột trong DataFrame (`spark_df`).  Đặc trưng trễ là các giá trị của một cột ở các thời điểm *trước đó*.  Chúng thường rất hữu ích trong các bài toán dự đoán chuỗi thời gian (time series forecasting), nơi mà giá trị trong quá khứ có thể giúp dự đoán giá trị trong tương lai.

In [18]:
#Create lag to train the model
target_cols = [ "ind_ahor_fin_ult1",
    "ind_aval_fin_ult1",
    "ind_cco_fin_ult1",
    "ind_cder_fin_ult1",
    "ind_cno_fin_ult1",
    "ind_ctju_fin_ult1",
    "ind_ctma_fin_ult1",
    "ind_ctop_fin_ult1",
    "ind_ctpp_fin_ult1",
    "ind_deco_fin_ult1",
    "ind_deme_fin_ult1",
    "ind_dela_fin_ult1",
    "ind_ecue_fin_ult1",
    "ind_fond_fin_ult1",
    "ind_hip_fin_ult1",
    "ind_plan_fin_ult1",
    "ind_pres_fin_ult1",
    "ind_reca_fin_ult1",
    "ind_tjcr_fin_ult1",
    "ind_valo_fin_ult1",
    "ind_viv_fin_ult1",
    "ind_nomina_ult1",
    "ind_nom_pens_ult1",
    "ind_recibo_ult1"]

w = Window.partitionBy("ncodpers").orderBy("month_int")

lag_months =[1,2,3,4,5,6]
for lag in lag_months:
    for col in target_cols:
        spark_df = spark_df.withColumn(f"lag_{lag}_{col}", F.lag(F.col(col),lag).over(w))

# Tạo độ trễ (Lag Features) trong PySpark

Đoạn mã PySpark này tạo các *đặc trưng trễ* (lag features) cho một tập hợp các cột trong DataFrame (`spark_df`).  Đặc trưng trễ là các giá trị của một cột ở các thời điểm *trước đó*.  Chúng thường rất hữu ích trong các bài toán dự đoán chuỗi thời gian (time series forecasting), nơi mà giá trị trong quá khứ có thể giúp dự đoán giá trị trong tương lai.

In [ ]:
# Define a window partitioned by customer and ordered by date
w = Window.partitionBy("ncodpers").orderBy("month_int")

# Compute the previous month value to calculate the gap
spark_df = spark_df.withColumn("prev_month", F.lag("month_int").over(w))
spark_df = spark_df.withColumn("month_diff", 
                   F.when(F.col("prev_month").isNotNull(), 
                          F.col("month_int") - F.col("prev_month"))
                    .otherwise(0))

# Flag a gap if the difference is not equal to 1 (and ignore the first record)
spark_df = spark_df.withColumn("gap_flag", 
                   F.when((F.col("month_diff") != 1) & F.col("prev_month").isNotNull(), 1)
                    .otherwise(0))

# Xuất dữ liệu đã làm sạch sang file Parquet trong PySpark

Đoạn mã PySpark này thực hiện việc lưu trữ DataFrame (`spark_df`) đã được làm sạch và xử lý vào một file Parquet.  Parquet là một định dạng file cột (columnar storage format) phổ biến, được tối ưu hóa cho các hệ thống phân tích dữ liệu lớn như Spark.

In [ ]:
#Export cleaned data to .parquet file to train train the model
# spark_df.printSchema()
spark_df.write.mode("overwrite").parquet("train_test.parquet")